# Model eval: WBGT in models v. ISIMIP obs get matching periods in each model

For periods 1990-2019, 2000-2019, 2010-2019 in obs, what is corresponding period in each model (for model evaluation)

In [1]:
# necessary: netcdf4-python, regionmask, (xarray, geopandas), openpyxl?, cartopy?, statsmodels?

import numpy as np
import pandas as pd
import os, glob, re 
import math
import xarray as xr
import netCDF4
import time
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
%matplotlib inline

# My settings and functions
from settings_ana import *
start_message() 

from functions_ana import *
print(datetime.now(), 'fxns imported')

flags: {'models': 'ISIMIP3b', 'experiment': None, 'metric': 'WBGT28', 'method': None, 'time_method': None, 'shift_sigma': False, 'shift_period': None, 'shift_loglike': None, 'chunk_version': None, 'var': 'wbgt'}
var: wbgt
VARs: None
indir_p: /data/brussel/vo/000/bvo00012/data/dataset/ISIMIP/ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/
indir_s: /data/brussel/vo/000/bvo00012/data/dataset/ISIMIP/ISIMIP3b/SecondaryInputData/climate/atmosphere/bias-adjusted/global/daily/
indir_obs: /data/brussel/vo/000/bvo00012/data/dataset/ISIMIP/ISIMIP3a/InputData/climate/atmosphere/obsclim/global/daily/historical/
indir_counterclim: /data/brussel/vo/000/bvo00012/data/dataset/ISIMIP/ISIMIP3a/InputData/climate/atmosphere/counterclim/global/daily/historical/
GCMs: ['CanESM5', 'CNRM-CM6-1', 'GFDL-ESM4', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'EC-Earth3', 'UKESM1-0-LL', 'MPI-ESM1-2-HR', 'CNRM-ESM2-1']
datasets: ['GSWP3-W5E5', '20CRv3-ERA5', '20CRv3-W5E5']
lat_chunk: 90
lon_chunk: 120
tim

In [9]:
outdir=os.path.join(outdirs, 'output_gmt-mapping')

In [3]:
df_gmst_mod = merge_model_gmst(GCMs, dir_gmst_models)
df_gmst_mod

,CanESM5,CNRM-CM6-1,GFDL-ESM4,IPSL-CM6A-LR,MIROC6,MRI-ESM2-0,EC-Earth3,UKESM1-0-LL,MPI-ESM1-2-HR,CNRM-ESM2-1
year,,,,,,,,,,
1850,0.096275,0.07598,-0.094941,-0.184196,-0.068196,0.062608,-0.087824,-0.053196,0.009157,0.097922
1851,-0.149725,0.03998,-0.151941,-0.016196,-0.095196,0.087608,0.015176,-0.045196,0.025157,0.266922
1852,-0.101725,0.00998,0.027059,-0.032196,-0.079196,0.021608,0.070176,0.042804,0.199157,0.184922
1853,0.038275,0.03198,0.061059,0.036804,-0.019196,-0.052392,-0.111824,0.104804,0.301157,0.061922
1854,-0.075725,0.03098,-0.113941,0.107804,0.191804,-0.107392,-0.159824,0.030804,0.010157,0.058922
...,...,...,...,...,...,...,...,...,...,...
2096,6.090275,4.75998,3.465059,5.281804,3.370804,3.743608,4.563176,5.822804,3.601157,4.443922
2097,6.278275,4.75398,3.466059,5.512804,3.036804,3.795608,4.910176,6.047804,3.600157,4.466922
2098,6.364275,4.95798,3.442059,5.356804,3.112804,3.624608,5.041176,6.080804,3.676157,4.525922


In [4]:

# annual obs warming
df_gmst_obs_annual = pd.read_csv(os.path.join('../../data/gmst/gmst-obs/forster2023/annual_averages.csv')).rename(
    columns={'timebound_lower':'year'}).set_index('year')[['gmst']]

df_gmst_obs_annual

,gmst
year,
1850,-0.053137
1851,0.056863
1852,0.084363
1853,0.049363
1854,0.049363
...,...
2018,1.109363
2019,1.236863
2020,1.269363


In [5]:
#obs_path = os.path.join('../../data/gmst/gmst-obs/forster2023/annual_averages.csv')
obs_path = observed_warming_path_annual

da_10 = calc_warming_periods_models_all_years(GCMs, 
                                          dir_gmst_models, 
                                          observed_warming_path=obs_path,
                                          method='window',
                                          window=10,
                                          centered=False,
                                          min_periods=1,
                                          flatten=False).sel(year=2019)

da_20 = calc_warming_periods_models_all_years(GCMs, 
                                          dir_gmst_models, 
                                          observed_warming_path=obs_path,
                                          method='window',
                                          window=20,
                                          centered=False,
                                          min_periods=1,
                                          flatten=False).sel(year=2019)

da_30 = calc_warming_periods_models_all_years(GCMs, 
                                          dir_gmst_models, 
                                          observed_warming_path=obs_path,
                                          method='window',
                                          window=30,
                                              centered=False,
                                          min_periods=1,
                                          flatten=False).sel(year=2019)

In [11]:
da_10.to_pandas()

feature,year_mod,temp
model,,
obs,2019,1.071304
CanESM5,2004,1.085475
CNRM-CM6-1,2021,1.08708
GFDL-ESM4,2029,1.069159
IPSL-CM6A-LR,2009,1.058504
MIROC6,2034,1.096404
MRI-ESM2-0,2021,1.076008
EC-Earth3,2013,1.055976
UKESM1-0-LL,2019,1.086504


In [12]:
da_20.to_pandas()

feature,year_mod,temp
model,,
obs,2019,0.965929
CanESM5,2008,0.971625
CNRM-CM6-1,2021,0.96888
GFDL-ESM4,2029,0.962609
IPSL-CM6A-LR,2010,0.964854
MIROC6,2034,0.956754
MRI-ESM2-0,2022,0.962758
EC-Earth3,2016,0.948576
UKESM1-0-LL,2021,0.962754


In [13]:
da_30.to_pandas()

feature,year_mod,temp
model,,
obs,2019,0.865971
CanESM5,2009,0.874408
CNRM-CM6-1,2020,0.858314
GFDL-ESM4,2029,0.859925
IPSL-CM6A-LR,2011,0.869971
MIROC6,2033,0.862037
MRI-ESM2-0,2024,0.878141
EC-Earth3,2020,0.859476
UKESM1-0-LL,2023,0.848204


In [10]:
df_join = pd.DataFrame(
    data = {'10_yr': da_10.to_pandas().year_mod, 
            '20_yr': da_20.to_pandas().year_mod, 
            '30_yr': da_30.to_pandas().year_mod},
    index=da_10.to_pandas().index
)
   
#df_join.to_csv(os.path.join(outdir,'df_gmt_mapping_endyear2019_eval_allmodels.csv'))

df_join 

,10_yr,20_yr,30_yr
model,,,
obs,2019,2019,2019
CanESM5,2004,2008,2009
CNRM-CM6-1,2021,2021,2020
GFDL-ESM4,2029,2029,2029
IPSL-CM6A-LR,2009,2010,2011
MIROC6,2034,2034,2033
MRI-ESM2-0,2021,2022,2024
EC-Earth3,2013,2016,2020
UKESM1-0-LL,2019,2021,2023
